<a href="https://colab.research.google.com/github/Shifuumi/neuro-workshop-/blob/main/NLP_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Natural Language Processing </a>

## Assignment: K Nearest Neighbors Model for the IMDB Movie Review Dataset

For the final project, build a K Nearest Neighbors model to predict the sentiment (positive or negative) of movie reviews. The dataset is originally hosted here: http://ai.stanford.edu/~amaas/data/sentiment/

Use the notebooks from the class and implement the model, train and test with the corresponding datasets.

You can follow these steps:
1. Read training-test data (Given)
2. Train a KNN classifier (Implement)
3. Make predictions on your test dataset (Implement)

__You can use the KNN Classifier from here: https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html__

## 1. Reading the dataset

We will use the __pandas__ library to read our dataset. 

#### __Training data:__
Let's read our training data. Here, we have the text and label fields. Labe is 1 for positive reviews and 0 for negative reviews.

In [1]:
import pandas as pd

train_df = pd.read_csv('https://raw.githubusercontent.com/aws-samples/aws-machine-learning-university-accelerated-nlp/master/data/final_project/imdb_train.csv', header=0)
train_df.head()

,text,label
0,This movie makes me want to throw up every tim...,0
1,Listening to the director's commentary confirm...,0
2,One of the best Tarzan films is also one of it...,1
3,Valentine is now one of my favorite slasher fi...,1
4,No mention if Ann Rivers Siddons adapted the m...,0


#### __Test data:__

In [3]:
import pandas as pd

test_df = pd.read_csv('https://raw.githubusercontent.com/aws-samples/aws-machine-learning-university-accelerated-nlp/master/data/final_project/imdb_test.csv', header=0)
test_df.head()

,text,label
0,What I hoped for (or even expected) was the we...,0
1,Garden State must rate amongst the most contri...,0
2,There is a lot wrong with this film. I will no...,1
3,"To qualify my use of ""realistic"" in the summar...",1
4,Dirty War is absolutely one of the best politi...,1


## 2. Train a KNN Classifier
Here, you will apply pre-processing operations we covered in the class. Then, you can split your dataset to training and validation here. For your first submission, you will use __K Nearest Neighbors Classifier__. It is available [here](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html).

In [7]:
# import lib
import nltk

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [30]:
#cleaning process

import nltk, re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize

stop = stopwords.words('english')
snow = SnowballStemmer('english')

excluding = ['against', 'not', 'don', "don't",'ain', 'aren', "aren't", 'couldn', "couldn't",
             'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 
             'haven', "haven't", 'isn', "isn't", 'mightn', "mightn't", 'mustn', "mustn't",
             'needn', "needn't",'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', 
             "weren't", 'won', "won't", 'wouldn', "wouldn't"]

stop_words = [word for word in stop if word not in excluding]

def process_final(sent_list) :
    final_list = []
    for sent in sent_list :
      if not isinstance(sent, str) :
        sent = ""
      else:
        tmp_sent = []
        sent = sent.lower()
        sent = sent.strip()
        sent = re.sub('\s+', ' ', sent)
        sent = re.compile('<.*?>').sub('', sent)

        for word in word_tokenize(sent) :
          if not word.isnumeric() and len(word) > 2 and word not in stop_words :
            tmp_sent.append(snow.stem(word))
        final = " ".join(tmp_sent)
        final_list.append(final)

    return final_list

In [56]:
#split the data for training and testing

from sklearn.model_selection import train_test_split

X = train_df[["text"]]
Y = train_df["label"]
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.15, random_state=123)

In [57]:
#try processing the training set
import traceback

try :
  train_text_list = process_final(X_train["text"].tolist())
  val_text_list = process_final(X_val["text"].tolist())
except Exception:
    traceback.print_exc()


In [58]:
#knn algo
import gensim
from gensim.models import Word2Vec
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn import set_config

w2v = gensim.models.Word2Vec()
pipeline = Pipeline([('text_vect', CountVectorizer(binary=True, max_features=10)), ('knn', KNeighborsClassifier())])
set_config(display='diagram')
pipeline

X_train = train_text_list
X_val = val_text_list

pipeline.fit(X_train, Y_train.values)

Pipeline(steps=[('text_vect', CountVectorizer(binary=True, max_features=10)),
                ('knn', KNeighborsClassifier())])

## 3. Make predictions on your test dataset

Once we select our best performing model, we can use it to make predictions on the test dataset. You can simply use __.fit()__ function with your training data to use the best performing K value and use __.predict()__ with your test data to get your test predictions.

In [59]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

val_predictions = pipeline.predict(X_val)
print(confusion_matrix(Y_val.values, val_predictions))
print(classification_report(Y_val.values, val_predictions))
print("Accuracy (validation):", accuracy_score(Y_val.values, val_predictions))

[[1004  853]
 [ 948  945]]
              precision    recall  f1-score   support

           0       0.51      0.54      0.53      1857
           1       0.53      0.50      0.51      1893

    accuracy                           0.52      3750
   macro avg       0.52      0.52      0.52      3750
weighted avg       0.52      0.52      0.52      3750

Accuracy (validation): 0.5197333333333334
